In [2]:
import os
import torch
import numpy as np
import glob
import torch.nn as nn

In [8]:
import torch
import os
import numpy as np
import glob
import torch.nn as nn
from torchvision import datasets as dts
from torchvision.transforms import transforms
from torch.utils.data import DataLoader #let op dubbele hoofdletter
from torch.optim import Adam
#from torch.autograd import Variable
import torchvision
import pathlib
from pathlib import Path

In [9]:
# doet cuda het?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
traindt = dts.MNIST(
    root = 'data',
    train = True,                             
    transform = transforms.ToTensor(), 
    download = True,            
)
testdt = dts.MNIST(
    root = 'data', 
    train = False, 
    transform = transforms.ToTensor()
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



NameError: name 'ToTensor' is not defined

In [6]:
# transforms instellen (hier zou je de pipeline aanmaken stel dat je dat wil, vind het zelf onhandig en onvoorspelbaar)
transformer = transforms.Compose([
    transforms.Resize((350,350)),
    transforms.ToTensor(),#tensortjes maken zodat torch er mee om kan gaan (zet ook pixel waarden tussen 0 en 1)
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5]).cuda()
])#deze termen zorgen voor transformatie naar -1 , 1 dit zou beter moeten zijn voor het trainen, ik weet niet waarom
    
    


In [7]:
# data inladen (dataloader instellen)
train_path = 'data_augmented_colorLUV2RGB'

train_loader= DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size=32, shuffle=True
)

test_path = 'MNIST_dataset'

test_loader= DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    batch_size=32, shuffle=True
)

FileNotFoundError: [Errno 2] No such file or directory: 'data_augmented_colorLUV2RGB'

In [ ]:
# klassen/labels halen
root = Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
# gelukt?
print(classes)

In [ ]:
# model bouwen 
# input shape wordt : (32,3,350,350) (batchsize, rgb, dimensions)
# batch normalization komt er op neer dat je verspreiding van de range van data steeds weer inkrimpt om het rekenen binnen perken te houden https://arxiv.org/abs/1502.03167

class ConvNet(nn.Module):
    def __init__(self,num_classes=4):
        super(ConvNet,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=128,kernel_size=3,stride=1,padding='valid')
        self.bn1 = nn.BatchNorm2d(num_features=128)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        self.conv2 = nn.Conv2d(in_channels=128,out_channels=64,kernel_size=3,stride=1,padding='valid')
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding='valid')
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        self.conv4 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,stride=1,padding='valid')
        self.bn4 = nn.BatchNorm2d(num_features=32)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        
        self.conv5 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding='valid')
        self.bn5 = nn.BatchNorm2d(num_features=32)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2)
        
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=2592, out_features = 256,device=device)
        self.dropout1 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=256, out_features = num_classes,device=device)
        self.dropout2 = nn.Dropout(0.25)

        
    def forward(self,input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool1(output)
        
        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool3(output)
        
        output = self.conv4(output)
        output = self.bn4(output)
        output = self.relu4(output)
        output = self.pool4(output)
        
        output = self.conv5(output)
        output = self.bn5(output)
        output = self.relu5(output)
        output = self.pool5(output)
        
        # output = output.view(-1,32,9,9)
        
        output = self.flat(output)
        output = self.dropout1(self.fc1(output))
        output = self.dropout2(self.fc2(output))
        
        return output
        
        # als het goed is komt hier een matrix uit

In [ ]:
# model op de hardware laden
model = ConvNet(num_classes=4).to(device)


In [ ]:
#optimaaaizer en loss function
optimizer = Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function = nn.CrossEntropyLoss().cuda()


In [ ]:
num_epochs = 100
 

In [ ]:
# checken hoeveel bestanden we hebben voor train en test (alle bestanden)
train_count = len(glob.glob(train_path+'/**/*.*'))
test_count = len(glob.glob(test_path+'/**/*.*'))

In [ ]:
print(train_count, test_count)


In [ ]:
list(map(lambda x: x.device, model.parameters()))

In [ ]:
# trainen en opslaan

best_accuracy = 0.0

for epoch in range(num_epochs):
    
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
    # zorgen dat de bestanden via de gpu gaan als die beschikbaar is
            images = images.cuda()
            labels = labels.cuda()
# zorgt dat de optimizer niet verward raakt door mixen van resultaat per batch        
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data,1)
    
        train_accuracy += int(torch.sum(prediction==labels.data))

    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    #test set evalueren
    model.eval()
    
    test_accuracy = 0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        outputs = model(images)
        _, prediction = torch.max(outputs.data,1)
        test_accuracy += int(torch.sum(prediction==labels.data))
        
    test_accuracy = test_accuracy/test_count

    print('Epoch: '+str(epoch)+' Train Loss: '+str(int(train_loss))+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    # model opslaan als beste is van getraint        
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy

In [ ]:
print(best_accuracy)
# print(model.get_parameters())
# print(model)
# list(map(lambda x: x.device, model.parameters()))

In [ ]:
# Hoogst behaald: 0.7333333333333333 - standaard dataset
#  - gelijk aantal per klasse, flipped